<a href="https://colab.research.google.com/github/eftekhar-hossain/SKBI_Training/blob/main/Session_5_(Incubator_Training).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install scikit-learn==0.21.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sklearn
print(sklearn.__version__)

0.21.3


In [5]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from tensorflow.keras.preprocessing.text import Tokenizer
np.random.seed(42)

CPU times: user 3.73 s, sys: 469 ms, total: 4.2 s
Wall time: 6.83 s


In [ ]:
!pip install bnlp_toolkit # installing for Bangla toolkit

In [2]:
# Download the data
!wget -O Emotion.csv https://www.dropbox.com/s/pjzwlxbmm7sq4sf/emotion.csv?dl=0

--2022-06-29 12:06:58--  https://www.dropbox.com/s/pjzwlxbmm7sq4sf/emotion.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pjzwlxbmm7sq4sf/emotion.csv [following]
--2022-06-29 12:06:58--  https://www.dropbox.com/s/raw/pjzwlxbmm7sq4sf/emotion.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce9a8a5e1349da94c1d5639b335.dl.dropboxusercontent.com/cd/0/inline/BoH1r72Hlu9iBoPSqaL-rONmUFM6UYNQCBrybI9xfrYNDdxg83ObaiWKdwkLwK-LupczrmFW2TFcSxk2gl3TcB0bBtfGiTrv-PhtJKbgyFVGisjo8ylx0XTURcTYJ2HYkEDDXUa3jujQ0mXHrUh9FQGKvlbGJGWcvsgAhSCOncoZ0g/file# [following]
--2022-06-29 12:06:59--  https://uce9a8a5e1349da94c1d5639b335.dl.dropboxusercontent.com/cd/0/inline/BoH1r72Hlu9iBoPSqaL-rONmUFM6UYNQCBrybI9xfrYNDdxg83ObaiWKdwkLwK-Lup

In [6]:
# %%time 
data = pd.read_csv("Emotion.csv")
data = data.dropna()

In [7]:
print("Size of the total data==> ", len(data))
class_names= data.classes.unique() ## Get the unique values in a specific column
print(class_names)

Size of the total data==>  29290
['joy' 'sadness' 'surprise' 'disgust' 'anger' 'fear']


In [8]:
data['training_label']=data.classes.replace({'joy':0,'sadness':1,'surprise':2,'disgust':3,'anger':4,'fear':5})
data.head(20)

,TEXT,classes,training_label
0,benson এ একটা জোস জিনিশ বার করল! benson fine c...,joy,0
1,ভাই রে ভাই....কিরণ দত্ত একটা চিজ....ভালোবাসা ভাই।,joy,0
2,"দাদা তোমাকে খুব ভালোবাসি, সবসময় তোমার ভিডিও দে...",joy,0
3,ওভাই....কোন অঞ্চলের বাংলা ভাষা ?? না মানে ...,joy,0
4,তোরে কে আঘাত দিয়েই আসলেই রিসেন্টলি আঘাত পাইসো...,joy,0
5,ডাকাত বলসে হাহা আমার ক্রাশ কে ...আম্মাজান কি ...,joy,0
6,"রাজা খুব দ্রুত মারা গেলেন, তার সাদা আলখাল্লা র...",sadness,1
7,"পসানিয়াস, রাজার দেহরক্ষীদের অন্যতম সদস্য হঠাৎ ...",surprise,2
8,“এত ভালো একটা ঘোড়াকে এভাবে ছেড়ে দেওয়া হচ্ছে! এ...,disgust,3
9,“বড়দের সমালোচনা করার মতো তুমি কে? তোমার কি মনে...,disgust,3


In [11]:
%%time

from bnlp.corpus import stopwords, punctuations
from nltk import word_tokenize
## Data samples before cleaning
for i in range(10):
  print(data['TEXT'][i])
## Cleaning the data. Removing newlines, unnecessary symbols

punctuations = '''“”!()-[]{};:'"\,<>./?@#$%^&*_~�।’‘1234567890''' #we can add suitable extra punctuation all the time
def remove_punctuation(d):
    review = d.replace('\n', '')
    no_punct = ""
    for char in review:
      if char not in punctuations:
         no_punct = no_punct + char
    return no_punct

def remove_stopwords(d):
  text_tokens = word_tokenize(d)
  bn_stopwords = stopwords() 
  tokens_without_sw = [word for word in text_tokens if not word in bn_stopwords]
  ls = ""
  for w in tokens_without_sw:
    ls = ls +" "+w
  return ls

data['cleaned'] = data['TEXT'].apply(remove_punctuation)
#data['cleaned'] = data['cleaned'].apply(remove_stopwords)

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


benson এ একটা জোস জিনিশ বার করল! benson fine cut  benson and benson light এর মাঝামাঝি! নয় কড়া নয় নয় হালকা! প্রথম দিনেই দিওয়ানা হয় পরলাম  যায় হোক আমার ভাল্লাগসে! 
ভাই রে ভাই....কিরণ দত্ত একটা চিজ....ভালোবাসা ভাই।
দাদা তোমাকে খুব ভালোবাসি, সবসময় তোমার ভিডিও দেখি এবং খুব মজা পাই। তোমার পাগলা ফ্যান দাদা। বাংলাদেশ থেকে বলছি
 ওভাই....কোন  অঞ্চলের বাংলা  ভাষা  ?? না মানে বুঝতে পারসিনাঃ  ভাই ..জানতে মন চাইলো ...
 তোরে কে আঘাত দিয়েই আসলেই রিসেন্টলি আঘাত পাইসোস এর আগেও পাইসোস 
ডাকাত বলসে হাহা  আমার ক্রাশ কে ...আম্মাজান কি করলেন..চুলতো এবার গেসে 
রাজা খুব দ্রুত মারা গেলেন, তার সাদা আলখাল্লা রক্তে ভিজে লাল হয়ে উঠেছে। রাজকন্যার রাজকীয় বিবাহ অনুষ্ঠানের আনন্দ-মধুর পরিবেশ সহসাই রূপ নিয়েছে গভীর শোক আর আর্তনাদে।
পসানিয়াস, রাজার দেহরক্ষীদের অন্যতম সদস্য হঠাৎ করেই আলখাল্লার ভেতর থেকে ছুরি বের করে রাজার বুকে বিঁধে দিয়েই দৌড় দিয়েছিলো বাগানের দিকে, ঘোড়ায় ওঠার জন্য।
“এত ভালো একটা ঘোড়াকে এভাবে ছেড়ে দেওয়া হচ্ছে! এরকম অনভিজ্ঞ আর দুর্বল লোক থাকলে এরকমই হয়,” পেছন থেকে চিৎকার করে উঠলেন যুবরাজ।
“বড়দের সমালোচনা করার 

In [12]:
data.head(15)

,TEXT,classes,training_label,cleaned
0,benson এ একটা জোস জিনিশ বার করল! benson fine c...,joy,0,benson এ একটা জোস জিনিশ বার করল benson fine cu...
1,ভাই রে ভাই....কিরণ দত্ত একটা চিজ....ভালোবাসা ভাই।,joy,0,ভাই রে ভাইকিরণ দত্ত একটা চিজভালোবাসা ভাই
2,"দাদা তোমাকে খুব ভালোবাসি, সবসময় তোমার ভিডিও দে...",joy,0,দাদা তোমাকে খুব ভালোবাসি সবসময় তোমার ভিডিও দেখ...
3,ওভাই....কোন অঞ্চলের বাংলা ভাষা ?? না মানে ...,joy,0,ওভাইকোন অঞ্চলের বাংলা ভাষা না মানে বুঝতে ...
4,তোরে কে আঘাত দিয়েই আসলেই রিসেন্টলি আঘাত পাইসো...,joy,0,তোরে কে আঘাত দিয়েই আসলেই রিসেন্টলি আঘাত পাইসো...
5,ডাকাত বলসে হাহা আমার ক্রাশ কে ...আম্মাজান কি ...,joy,0,ডাকাত বলসে হাহা আমার ক্রাশ কে আম্মাজান কি করল...
6,"রাজা খুব দ্রুত মারা গেলেন, তার সাদা আলখাল্লা র...",sadness,1,রাজা খুব দ্রুত মারা গেলেন তার সাদা আলখাল্লা রক...
7,"পসানিয়াস, রাজার দেহরক্ষীদের অন্যতম সদস্য হঠাৎ ...",surprise,2,পসানিয়াস রাজার দেহরক্ষীদের অন্যতম সদস্য হঠাৎ ক...
8,“এত ভালো একটা ঘোড়াকে এভাবে ছেড়ে দেওয়া হচ্ছে! এ...,disgust,3,এত ভালো একটা ঘোড়াকে এভাবে ছেড়ে দেওয়া হচ্ছে এরক...
9,“বড়দের সমালোচনা করার মতো তুমি কে? তোমার কি মনে...,disgust,3,বড়দের সমালোচনা করার মতো তুমি কে তোমার কি মনে হ...


##Visualize tf-idf and count vectorizer

In [13]:
tf_idf = TfidfVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split(),  use_idf = True)
tf_corpus = tf_idf.fit_transform(data.cleaned[:5])
cv = CountVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split())
cv_corpus = cv.fit_transform(data.cleaned[:5])

In [14]:
#Here first parameter is the number of documents and second parameter is the number of features
for i in range(5):
  print(data.cleaned[i])
print(tf_corpus.shape, cv_corpus.shape)
print(tf_idf.vocabulary_) ## Returns the vocabulary with index
print(cv.vocabulary_)
print(tf_idf.get_feature_names()) ## Return the feature words
print(cv.get_feature_names())
print(tf_idf.get_params()) ## Return the parameter values of the vectorizer
print(cv.get_params())

benson এ একটা জোস জিনিশ বার করল benson fine cut  benson and benson light এর মাঝামাঝি নয় কড়া নয় নয় হালকা প্রথম দিনেই দিওয়ানা হয় পরলাম  যায় হোক আমার ভাল্লাগসে 
ভাই রে ভাইকিরণ দত্ত একটা চিজভালোবাসা ভাই
দাদা তোমাকে খুব ভালোবাসি সবসময় তোমার ভিডিও দেখি এবং খুব মজা পাই তোমার পাগলা ফ্যান দাদা বাংলাদেশ থেকে বলছি
 ওভাইকোন  অঞ্চলের বাংলা  ভাষা   না মানে বুঝতে পারসিনাঃ  ভাই জানতে মন চাইলো 
 তোরে কে আঘাত দিয়েই আসলেই রিসেন্টলি আঘাত পাইসোস এর আগেও পাইসোস 
(5, 65) (5, 65)
{'benson': 1, 'এ': 10, 'একটা': 11, 'জোস': 23, 'জিনিশ': 22, 'বার': 46, 'করল': 15, 'fine': 3, 'cut': 2, 'and': 0, 'light': 4, 'এর': 13, 'মাঝামাঝি': 56, 'নয়': 35, 'কড়া': 17, 'হালকা': 62, 'প্রথম': 41, 'দিনেই': 31, 'দিওয়ানা': 30, 'হয়': 64, 'পরলাম': 36, 'যায়': 58, 'হোক': 63, 'আমার': 8, 'ভাল্লাগসে': 51, 'ভাই': 48, 'রে': 60, 'ভাইকিরণ': 49, 'দত্ত': 28, 'চিজভালোবাসা': 20, 'দাদা': 29, 'তোমাকে': 24, 'খুব': 18, 'ভালোবাসি': 50, 'সবসময়': 61, 'তোমার': 25, 'ভিডিও': 53, 'দেখি': 33, 'এবং': 12, 'মজা': 54, 'পাই': 37, 'পাগলা': 39, 'ফ্যান': 42, 'বাংলাদেশ':

In [15]:
# Convert sparse matrix to dataframe
tf_corpus = pd.DataFrame.sparse.from_spmatrix(tf_corpus)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in tf_idf.vocabulary_.items()}
# Rename each column using the mapping
for col in tf_corpus.columns:
    tf_corpus.rename(columns={col: col_map[col]}, inplace=True)
tf_corpus

,and,benson,cut,fine,light,অঞ্চলের,আগেও,আঘাত,আমার,আসলেই,...,মন,মাঝামাঝি,মানে,যায়,রিসেন্টলি,রে,সবসময়,হালকা,হোক,হয়
0,0.145399,0.581595,0.145399,0.145399,0.145399,0.000000,0.000000,0.000000,0.145399,0.000000,...,0.000000,0.145399,0.000000,0.145399,0.000000,0.000000,0.0,0.145399,0.145399,0.145399
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.371273,0.0,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.292968,0.000000,0.000000,0.000000,0.000000,...,0.292968,0.000000,0.292968,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.261257,0.522514,0.000000,0.261257,...,0.000000,0.000000,0.000000,0.000000,0.261257,0.000000,0.0,0.000000,0.000000,0.000000


In [16]:
# Convert sparse matrix to dataframe
cv_corpus = pd.DataFrame.sparse.from_spmatrix(cv_corpus)
# Save mapping on which index refers to which words
col_map = {v:k for k, v in cv.vocabulary_.items()}
# Rename each column using the mapping
for col in cv_corpus.columns:
    cv_corpus.rename(columns={col: col_map[col]}, inplace=True)
cv_corpus

,and,benson,cut,fine,light,অঞ্চলের,আগেও,আঘাত,আমার,আসলেই,...,মন,মাঝামাঝি,মানে,যায়,রিসেন্টলি,রে,সবসময়,হালকা,হোক,হয়
0,1,4,1,1,1,0,0,0,1,0,...,0,1,0,1,0,0,0,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,2,0,1,...,0,0,0,0,1,0,0,0,0,0


In [ ]:
corpus, labels = data.cleaned, data.training_label

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(corpus,labels,train_size = 0.8, test_size = 0.2,random_state =0)

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline


In [ ]:
%%time
clf = TfidfVectorizer(ngram_range=(1,1), min_df=1, tokenizer=lambda x: x.split(), use_idf=True)
vec = LogisticRegressionCV()
pipe = make_pipeline(clf, vec)
pipe.fit(X_train, y_train)

CPU times: user 1min 56s, sys: 2min 17s, total: 4min 13s
Wall time: 2min 31s


In [ ]:
from sklearn import metrics
def print_report(pipe):
    y_pred = pipe.predict(X_test)
    report = metrics.classification_report(y_test, y_pred,
        target_names=class_names)
    cm= confusion_matrix(y_test, y_pred)
    print(report)
    print(cm)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

print_report(pipe)

              precision    recall  f1-score   support

         joy       0.76      0.83      0.79      1565
     sadness       0.71      0.73      0.72      1361
    surprise       0.78      0.79      0.78      1076
     disgust       0.78      0.96      0.86       524
       anger       0.70      0.52      0.60       716
        fear       0.72      0.56      0.63       616

    accuracy                           0.74      5858
   macro avg       0.74      0.73      0.73      5858
weighted avg       0.74      0.74      0.74      5858

[[1295  103   77   22   35   33]
 [ 172  991   58   31   66   43]
 [ 111   51  846   19   15   34]
 [   4    8    5  505    0    2]
 [  70  148   41   61  372   24]
 [  63  101   53   11   42  346]]
accuracy: 0.743


In [17]:
!pip install eli5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 5.1 MB/s 
     |████████████████████████████████| 133 kB 60.6 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=78af6651bfbb807131660855afd4d9eb56c77b9c2eaee53dcec8103d2ae7113d
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is inco

In [ ]:
import eli5
eli5.show_weights(pipe, top=(10,10), target_names= class_names)

In [ ]:
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(class_names, y_pred):
        print("{:.3f} {}".format(prob, target))

doc = data.cleaned[2]
print(doc)
print_prediction(doc)

দাদা তোমাকে খুব ভালোবাসি সবসময় তোমার ভিডিও দেখি এবং খুব মজা পাই তোমার পাগলা ফ্যান দাদা বাংলাদেশ থেকে বলছি
0.885 joy
0.067 sadness
0.028 surprise
0.009 disgust
0.009 anger
0.001 fear


In [ ]:
from eli5.lime import TextExplainer

te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction( target_names = ['joy', 'sadness', 'surprise', 'disgust', 'anger', 'fear'])